In [1]:
import gc
import pandas as pd

In [2]:
train_active_df = pd.read_csv('../input/train_active.csv', usecols=['title', 'description'])
train_active_df = train_active_df.fillna('')
train_active_df['title_desc'] = train_active_df['title'].astype(str) + train_active_df['description'].astype(str)
train_active_df = train_active_df.drop(['title', 'description'], axis=1)

In [3]:
import os
import multiprocessing
cores = multiprocessing.cpu_count()

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(train_active_df.shape[0]):  
            desc = train_active_df.iloc[i,0]
            if desc != '':
                yield TaggedDocument(words=simple_preprocess(train_active_df.iloc[i,0]), tags=[i])
            else:
                continue

In [ ]:
%%time

if not os.path.exists('models/doc2vec.model'):
    print("start traing doc2vec model...")
    documents = MyDocs()
    doc2vec_model = Doc2Vec(dm=1, vector_size=100, window=8, min_count=40, workers=cores, epochs=10)
    doc2vec_model.build_vocab(documents)
    doc2vec_model.train(documents, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
    if not os.path.exists('models'): 
        os.makedirs('models')
        doc2vec_model.save('models/doc2vec.model')
    else:
        
        doc2vec_model.save('models/doc2vec.model')
else:
    doc2vec_model = Doc2Vec.load('models/doc2vec.model')

start traing doc2vec model...


In [11]:
del train_active_df
gc.collect()

1001

In [42]:
train_df = pd.read_csv('../input/train.csv', usecols=['title', 'description'])
train_df = train_df.fillna('')
train_df['title_desc'] = train_df['title'].astype(str) + train_df['description'].astype(str)
train_df = train_df.drop(['title', 'description'], axis=1)

In [34]:
print(train_df.iloc[0])

title_desc    Отдам бесплатноНа ангарском
Name: 0, dtype: object


In [43]:
vector_df = []
for i in range(train_df.shape[0]):
    vector_df.append(list(doc2vec_model.infer_vector(train_df.iloc[i])))

In [44]:
vector_names = [ 'd2v_des_'+ str(i) for i in range(100)]

In [45]:
vector_df = pd.DataFrame(vector_df, columns=vector_names)

In [46]:
vector_df.head()

,d2v_des_0,d2v_des_1,d2v_des_2,d2v_des_3,d2v_des_4,d2v_des_5,d2v_des_6,d2v_des_7,d2v_des_8,d2v_des_9,...,d2v_des_90,d2v_des_91,d2v_des_92,d2v_des_93,d2v_des_94,d2v_des_95,d2v_des_96,d2v_des_97,d2v_des_98,d2v_des_99
0,-0.000096,-0.000630,-0.000357,-0.002541,0.002342,0.004210,-0.002951,0.001396,0.000683,0.000909,...,0.002245,-0.001432,0.004529,0.001988,-0.003545,0.000194,0.001303,0.000449,-0.003420,0.000212
1,-0.003006,0.003880,-0.003489,-0.001837,-0.001414,-0.000688,-0.004706,0.003070,0.002686,-0.000754,...,0.002623,-0.002553,-0.000212,0.000602,0.000606,0.002775,-0.000732,-0.001920,0.003393,-0.003132
2,-0.003164,0.000502,0.001140,-0.002742,0.002900,0.003382,0.000870,0.001775,0.002504,0.004745,...,-0.001393,0.002211,-0.003565,-0.004867,-0.001775,0.003335,-0.000033,0.004824,-0.004838,0.001378
3,-0.004729,-0.001677,-0.000555,-0.002868,-0.004287,0.002324,-0.004298,-0.004555,0.003364,-0.002126,...,-0.000903,0.003889,-0.004152,-0.003814,0.003885,-0.003561,-0.004051,-0.002012,0.004686,-0.003631
4,-0.003339,-0.000116,-0.002446,0.003012,0.000756,0.004044,-0.003773,-0.000507,-0.002300,0.000783,...,-0.003446,0.001505,-0.002021,-0.003124,0.004838,0.003333,-0.001472,0.000731,0.002785,-0.003619


In [61]:
sum(train_df['title_desc'] == ' ')

0

In [40]:
# vector_df.to_csv('test_d2v.csv', index=False)

In [48]:
vector_df.to_pickle('../input/train_d2v.pkl')

In [59]:
doc2vec_model.docvecs.most_similar(123)

[(12407770, 0.887975811958313),
 (13212248, 0.8843343257904053),
 (8667468, 0.883764386177063),
 (8066923, 0.8831993341445923),
 (3939639, 0.8825210332870483),
 (6795815, 0.8818987011909485),
 (24590, 0.881564199924469),
 (10790629, 0.8815597295761108),
 (9441305, 0.8814713954925537),
 (1704067, 0.8811585307121277)]